In [2]:
import bokeh
import pandas as pd

In [102]:
from bokeh.models import ColumnDataSource, NumeralTickFormatter, OpenURL, TapTool, Slider, CustomJS
from bokeh.layouts import column
from bokeh.plotting import figure, show

In [124]:
#Import Data
path="CleanData2.csv"
df=pd.read_csv(path)
#Drop problem rows that dont play well with bokeh
df = df[~df['area_code'].str.startswith('C').fillna(False)].reset_index(drop=True)
df = df[~df['area_code'].str.startswith('US').fillna(False)].reset_index(drop=True)

df

C:\Users\dkace\anaconda3\envs\pythondata\lib\site-packages\IPython\core\interactiveshell.py:3186: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ID,area_code,State,County,quarter,year,establishments,total_employed,total_wages,taxable_wage,average_weekly_wage,%chg_lq_establishments,%chg_total_emp,%chg_lq_total_qtrly_wages,%chg_lq_taxable_wages,%chg_avg_wkly_wage
0,1,1001,Alabama,Autauga County,1,2020,1778,66384,238572054,97010816,4319,9.20,1.136000,1.190,0.452,1.024
1,2,1003,Alabama,Baldwin County,1,2020,13072,452026,1439657260,749596298,4103,4.58,0.780000,0.856,0.624,1.094
2,3,1005,Alabama,Barbour County,1,2020,1128,49570,160195240,80797450,4011,12.67,1.164000,1.216,0.626,1.052
3,4,1007,Alabama,Bibb County,1,2020,730,29156,110388552,48400892,4670,16.92,1.237333,1.264,0.680,1.036
4,5,1009,Alabama,Blount County,1,2020,1568,51336,152730444,72113324,3616,9.58,1.042667,1.124,0.796,1.034
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58924,58925,9999,Connecticut,Unknown Or Undefined,2,2022,18664,137734,1472699154,141550499,6816,3.20,0.292000,0.258,0.200,0.486
58925,58926,9999,Connecticut,Unknown Or Undefined,3,2022,21682,140604,1518109975,110302816,7389,3.11,0.298000,0.264,0.200,0.526
58926,58927,9999,Connecticut,Unknown Or Undefined,4,2022,23258,146868,1620191668,91946683,7319,3.08,0.299333,0.260,0.200,0.534
58927,58928,9999,Connecticut,Unknown Or Undefined,1,2023,24986,145389,2031326900,655044713,8352,3.08,0.305333,0.256,0.200,0.486


In [125]:
from bokeh.models import Div

header_div = Div(
    text="""
<h1>US Employment and Wage Data</h1>
<p>Data: Bureau of Labor Statistics, <a href="https://data.bls.gov/cew/apps/data_views/data_views.htm#tab=Tables" target="_blank">Quarterly Census Data</a></p target="_blank">
""",
    sizing_mode="stretch_both",
)

show(header_div)

In [126]:
df.columns

Index(['ID', 'area_code', 'State', 'County', 'quarter', 'year',
       'establishments', 'total_employed', 'total_wages', 'taxable_wage',
       'average_weekly_wage', '%chg_lq_establishments', '%chg_total_emp',
       '%chg_lq_total_qtrly_wages', '%chg_lq_taxable_wages',
       '%chg_avg_wkly_wage'],
      dtype='object')

In [138]:
from bokeh.models import FactorRange
source = ColumnDataSource(df)
from itertools import product

# Create a ColumnDataSource
bardf = df.groupby('total_wages').agg({
    'area_code': 'first',
    'State':'first',
    'County': 'first'
}).reset_index()
bardf = bardf.sort_values(by='total_wages', ascending=False)
source = ColumnDataSource(bardf)

# Extract unique states and counties
unique_states = df['State'].unique()

# Create FactorRange for x-axis with all unique combinations of State and County
x_values = list(product(unique_states))
x_range = FactorRange(*[' '.join(map(str, value)) for value in x_values])

# Set up the figure
plot = figure(
    x_range= x_range,
    title="Total Wages by County",
    tooltips=[("County", "@County"), ("Total Wages", "@total_wages")],
    toolbar_location=None,
    sizing_mode="scale_width",
)

# Add a vbar renderer
bars = plot.vbar(
    x='area_code',
    top='total_wages',
    nonselection_alpha=0.8,
    source=source,
    width=0.6
)

# Customize plot appearance
plot.yaxis.formatter = NumeralTickFormatter(format="0,0")
plot.xaxis.major_label_orientation = 45  # rotate x-axis labels

# Set up Slider widget
initial_value = 25  # Set the initial value to start with the top 25 counties
slider = Slider(
    start=1,
    end=50,
    value=initial_value,
    title="Number of entries to consider",
)

# Set up callback
custom_js = CustomJS(
    args={"source": source, "slider": slider, "x_range": plot.x_range},
    code="""
    const data = source.data;
    const new_range = x_range.factors.slice(0, slider.value);
    source.data['area_code'] = new_range;
    source.change.emit();
    """,
)


slider.js_on_change("value", custom_js)

# Assemble the layout
layout = column([slider, plot], sizing_mode="scale_width")

# Show the plot
show(layout)

In [120]:
bardf

,total_wages,area_code,State,County
58923,244955627952,36061,New York,New York County
58922,243225524674,36061,New York,New York County
58921,217379875740,36061,New York,New York County
58920,213170244852,36061,New York,New York County
58919,204164028928,36061,New York,New York County
...,...,...,...,...
4,788697,31117,Nebraska,McPherson County
3,788471,31117,Nebraska,McPherson County
2,763975,31005,Nebraska,Arthur County
1,737304,30069,Montana,Petroleum County
